In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
cols = ['remote_addr', 'time_iso8601', 'status', 'file', 'body_bytes_sent', 'request_time',
        'uuid', 'http_referer', 'http_user_agent', 'http_x_forwarded_for', 'pno', 'Id', 'auth_ret']

In [4]:
train = pd.read_table('../01-input/a.txt', header=None, names=['0'])
train.head()

,0
0,"[""6c455e37ea1f3f43443ecbe46bcdb4bb"", ""2021-03-..."
1,"[""64eaa6e58e5bd71d54dd7565481a2f1d"", ""2021-03-..."
2,"[""4d4c7b25b890132ee432bebab3940966"", ""2021-03-..."
3,"[""da177836163927897f26c718d1daf85f"", ""2021-03-..."
4,"[""affef57aacffb8aa6e911659e55f54b2"", ""2021-03-..."


In [ ]:
def process(x):
    remote_addr = map(lambda x: x[0], x)
    time_iso8601 = map(lambda x: x[1], x)
    status = map(lambda x: x[2], x)
    file = map(lambda x: x[3], x)
    body_bytes_sent = map(lambda x: x[4], x)
    request_time = map(lambda x: x[5], x)
    uuid = map(lambda x: x[6], x)
    http_referer = map(lambda x: x[7], x)
    http_user_agent = map(lambda x: x[8], x)
    http_x_forwarded_for = map(lambda x: x[9], x)
    pno = map(lambda x: x[10], x)
    Id = map(lambda x: x[11], x)
    auth_ret = map(lambda x: x[12], x)
    frame = pd.DataFrame({
        'remote_addr': remote_addr,
        'time_iso8601': time_iso8601,
        'status': status,
        'file': file,
        'body_bytes_sent': body_bytes_sent,
        'request_time': request_time,
        'uuid': uuid,
        'http_referer': http_referer,
        'http_user_agent': http_user_agent,
        'http_x_forwarded_for': http_x_forwarded_for,
        'pno': pno,
        'Id': Id,
        'auth_ret': auth_ret
    })
    return frame

df = train['0'].apply(process)
del train
gc.collect()
act_3 = pd.concat([i for i in df], ignore_index=False)

In [ ]:
train[0][:5]